![imagenes](logo.png)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 10)

import plotly.express as px

# Batch Gradient Descent

Trabajaremos un problema de [regresión lineal](https://github.com/scidatmath2020/Machine-Learning/blob/main/C01.2%20Regresi%C3%B3n%20lineal.ipynb)

Básicamente, $\beta_1$ y $\beta_2$ son los coeficientes que queremos encontrar tales que se minimice la función de coste $$L(\beta_1,\beta_2)=\frac{1}{2m}\sum_{j=1}^m(\beta_1x_{j1}+\beta_2x_{j2}-y_j)^2$$ por lo que $$\nabla(L(\beta_1,\beta_2))=\frac{1}{m}\sum_{j=1}^n(\hat{y_j}-y_j)\cdot x_j,$$ siendo $\hat{y_j}=\beta_1x_{j1}+\beta_2x_{j2}$

In [ ]:
Tabla = np.genfromtxt("C03_tabla.csv",delimiter=",")
coeficientes_objetivo = np.genfromtxt("C03_coeficientes_objetivo.csv",delimiter=",")

Lo bueno de usar este dataset es que sabemos exactamente cuales son los coeficientes $\beta_i$ que definen el modelo lineal que genera la variable objetivo

In [ ]:
Tabla

In [ ]:
df = pd.DataFrame(Tabla)
df.columns = ["x1","x2","y"]
df

In [ ]:
fig = px.scatter_3d(df, x='x1', y='x2', z='y')
fig.show()

In [ ]:
coeficientes_objetivo

In [ ]:
X = Tabla[:,0:2]
y = Tabla[:,2]

In [ ]:
X

Podemos obtener la variable objetivo mediante un producto escalar de los pesos con las variables independientes

In [ ]:
def predecir_batch(coeficientes, X):
    return coeficientes @ X.T

In [ ]:
y_predicciones = predecir_batch(coeficientes_objetivo, X)
y_predicciones[:10]

Comprobamos que dichas predicciones son exactamente iguales que la variable objetivo

In [ ]:
y_predicciones[:10]-y[:10]

Necesitamos una función de error, en este caso usaremos el **Error Cuadrático Medio** dividido entre 2, para que su derivada no tenga el 2

In [ ]:
def error_batch(y_pred, y_true):
    m = y_pred.shape[0]
    return (np.sum(y_pred - y_true)**2)/(2*m)

También necesitamos la derivada de la función de error.

In [ ]:
def derivada_error_batch(y_pred, y_true, x):
    m = y_pred.shape[0]
    return np.sum((y_pred - y_true)*x/m)

Para empezar el proceso generamos los coeficientes iniciales al azar

In [ ]:
coeficientes = np.random.random((X.shape[1],))
coeficientes

In [ ]:
coeficientes_objetivo

Ahora podemos predecir y calcular el error y la derivada del error

In [ ]:
y_pred = predecir_batch(coeficientes, X)

In [ ]:
error_batch(y_pred, y)

In [ ]:
derivada_error_batch(y_pred, y, X[:,0])

ya tenemos todo para implementar el descenso de gradiente batch

In [ ]:
def descenso_gradiente_batch(coeficientes, X, y):
    y_predicciones = predecir_batch(coeficientes, X)
    for i in range(coeficientes.shape[0]):
        coeficientes[i] = coeficientes[i]- STEP_SIZE * derivada_error_batch(y_predicciones, y, X[:,i])
    error = error_batch(y_predicciones, y)
    return coeficientes, error


Simplemente definimos un número de iteraciones y un tamaño de paso (tambien llamado **ratio de aprendizaje o learning rate**), iteraremos y en cada iteración modificaremos los parámetros del modelo en función del tamaño de paso.

In [ ]:
coeficientes_iteraciones = []
error_iteraciones = []

N_ITERACIONES = 200
STEP_SIZE = 0.02
coeficientes = np.random.random((X.shape[1],))
error = error_batch(coeficientes, X)
for i in range(N_ITERACIONES):
    coeficientes_iteraciones.append(coeficientes.copy())
    error_iteraciones.append(error)
    coeficientes, error = descenso_gradiente_batch(coeficientes, X, y)

coeficientes_iteraciones = np.array(coeficientes_iteraciones)

In [ ]:
coeficientes

Vemos que los coeficientes obtenidos se parecen mucho a los coeficientes objetivo.

In [ ]:
coeficientes_objetivo

In [ ]:
plt.plot(error_iteraciones)
plt.title("Evolución del error con el número de iteraciones");

In [ ]:
plt.plot(coeficientes_iteraciones[:,0], color="red")
plt.axhline(coeficientes_objetivo[0], color="red", linestyle="dashed")

plt.plot(coeficientes_iteraciones[:,1], color="blue")
plt.axhline(coeficientes_objetivo[1], color="blue", linestyle="dashed")

plt.xlabel("Numero de iteraciones")
plt.ylabel("Valor del coeficiente")

plt.title("Evolución de coeficientes con el número de iteraciones");

# Descenso de gradiente estocástico (SGD)

In [ ]:
def predecir_observacion(coeficientes, x):
    return coeficientes @ x.T

In [ ]:
X[0]

In [ ]:
predecir_observacion(coeficientes, X[0])

In [ ]:
y[0]

In [ ]:
def derivada_error_observacion(y_pred, y_true, x):
    return (y_pred - y_true) * x

In [ ]:
derivada_error_observacion(predecir_observacion(coeficientes, X[0]), y[0], X[0])

la definición de la iteracion de sgd es similar a la de batch, simplemente usando el error de observación en vez de el error total

In [ ]:
def descenso_gradiente_estocastico(coeficientes, x, y):
    y_predicciones = predecir_observacion(coeficientes, x)
    for i in range(coeficientes.shape[0]):
        coeficientes[i] = coeficientes[i]- STEP_SIZE * derivada_error_observacion(y_predicciones, y, x[i])
    return coeficientes

Ahora hacemos solo una iteración (podriamos hacer más), pero la iteramos para cada observación individual.

In [ ]:
coeficientes_iteraciones = []
error_iteraciones = []

STEP_SIZE = 0.02
coeficientes = np.random.random((X.shape[1],))
error = error_batch(coeficientes, X)

indice_aleatorio = np.random.permutation(X.shape[0])
for i in indice_aleatorio:
    error_iteraciones.append(error)
    coeficientes_iteraciones.append(coeficientes.copy())
    
    x_iteracion = X[i]
    y_iteracion = y[i]
    coeficientes = descenso_gradiente_estocastico(coeficientes,
                                                  x_iteracion,
                                                  y_iteracion)
    y_predicciones = predecir_batch(coeficientes, X)
    error = error_batch(y_predicciones, y)
    
coeficientes_iteraciones = np.array(coeficientes_iteraciones)

In [ ]:
coeficientes

In [ ]:
coeficientes_objetivo

In [ ]:
plt.plot(error_iteraciones)
plt.title("Evolución del error con el número de iteraciones");

In [ ]:
plt.plot(coeficientes_iteraciones[:,0], color="red")
plt.axhline(coeficientes_objetivo[0], color="red", linestyle="dashed")

plt.plot(coeficientes_iteraciones[:,1], color="blue")
plt.axhline(coeficientes_objetivo[1], color="blue", linestyle="dashed")

plt.xlabel("Numero de observaciones")
plt.ylabel("Valor del coeficiente")

plt.title("Evolución de coeficientes en SGD con el número de observaciones iteradas");

### SGD en scikit-learn

Scikit-learn tiene estimadores para regresión y clasificación basados en SGD, [SGDRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) y [SGDClassifier](from sklearn.linear_model import SGDClassifier)

Los parámetros más importantes para estos estimadores son:

- **loss**: La función de pérdidas a utilizar
- **learning_rate**: El tamaño de paso, también llamado learning rate (ratio de aprendizaje)
- **max_iter**: Número de iteraciones (también llamadas épocas)

In [ ]:
from sklearn.linear_model import SGDClassifier, SGDRegressor

In [ ]:
estimador_sgd = SGDRegressor(max_iter=10)
estimador_sgd.fit(X, y)

In [ ]:
estimador_sgd.predict(X)[:10]

Podemos ver los coeficientes que produce el estimador

In [ ]:
estimador_sgd.coef_

Y vemos que son muy similares a los coeficientes objetivo

In [ ]:
coeficientes_objetivo